In [8]:
# Usamos las librerias panda, requests y numpy
import pandas as pd
import requests
import numpy as np
from datetime import datetime
# leemos el fichero y lo guardamos en la variable df
df=pd.read_csv('gut_comps.csv',sep=";")
baseURL="https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchi/cids/JSON"
altURL="https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/xref/RegistryID/{registryID}/cids/JSON"
# Creamos la función getCidByInchI
def getCidByInchI(inchI):
    PARAMS = {'inchi':inchI}
    try:
        response = requests.get(url = baseURL, params = PARAMS)
        if(response.status_code==200):
            return response.json()["IdentifierList"]["CID"][0]
        else:
            return np.NA
    except:
        print(f'API Resquest ERROR {response.status_code}')
        response = requests.get(url = baseURL, params = PARAMS)
        if(response.status_code==200):
            return response.json()["IdentifierList"]["CID"][0]
        else:
            return np.NA
def getCidByHMDBId(HMDBId):
    try:
        response = requests.get(url = altURL.replace("{registryID}",HMDBId))
        if(response.status_code==200):
            return response.json()["IdentifierList"]["CID"][0]
        else:
            return np.NA
    except:
        print(f'API Resquest ERROR {response.status_code}')
        response = requests.get(url = altURL.replace("{registryID}",str(HMDBId)))
        if(response.status_code==200):
            return response.json()["IdentifierList"]["CID"][0]
        else:
            return np.NA
def checkDataNA(dataframe, colname, requestColname, altRequestColname):
    for i in range(0,len(dataframe[colname])):
        if(np.isnan(dataframe[colname][i])):
            dataframe[colname][i]=getCidByInchI(dataframe[requestColname][i])
        if(dataframe[colname][i]==0):
             dataframe[colname][i]=getCidByHMDBId(dataframe[altRequestColname][i])

In [ ]:
start_time = datetime.now()
# Recorremos la columna inchi y realizamos a tanta peticiones a la api como valores hay en la columna
df['cid'] = df.apply(lambda row:getCidByInchI(row['inchi']), axis = 1)
checkDataNA(df,'cid','inchi','hmdb_id')
# Guardamos el conjunto de datos en un fichero csv
df.to_csv("file_with_cids.csv", sep=';',index=False)
end_time=datetime.now()
print('Duration: {}'.format(end_time - start_time))